In [1]:
import configparser
import psycopg2
from sql_queries import copy_table_queries, insert_table_queries

def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        print("Executing copy:", query)
        try:
            cur.execute(query)
            conn.commit()
        except Exception as e:
            print(f"Error during copy: {e}")

def insert_tables(cur, conn):
    for query in insert_table_queries:
        print("Executing insert:", query)
        try:
            cur.execute(query)
            conn.commit()
        except Exception as e:
            print(f"Error during insert: {e}")

def main():
    config = configparser.ConfigParser()
    config.read('dwh.cfg')

    conn = None
    try:
        conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(
            config.get("CLUSTER", "HOST"),
            config.get("CLUSTER", "DB_NAME"),
            config.get("CLUSTER", "DB_USER"),
            config.get("CLUSTER", "DB_PASSWORD"),
            config.get("CLUSTER", "DB_PORT")))
        cur = conn.cursor()

        load_staging_tables(cur, conn)
        insert_tables(cur, conn)

    except Exception as e:
        print(f"Error connecting to Redshift: {e}")

    finally:
        if conn:
            cur.close()
            conn.close()
            print('ETL process completed and connection closed.')

if __name__ == "__main__":
    main()

Executing copy: 
COPY staging_events
FROM 's3://udacity-dend/log-data'
CREDENTIALS 'aws_iam_role=arn:aws:iam::891377030922:role/project-2-redshift-tbs89'
REGION 'us-west-2'
JSON 's3://udacity-dend/log_json_path.json'

Executing copy: 
COPY staging_songs
FROM 's3://udacity-dend/song-data'
CREDENTIALS 'aws_iam_role=arn:aws:iam::891377030922:role/project-2-redshift-tbs89'
REGION 'us-west-2'
JSON 'auto'

Executing insert: 
INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
SELECT TIMESTAMP 'epoch' + e.ts / 1000 * INTERVAL '1 second' AS start_time,
       e.userId AS user_id,
       e.level,
       s.song_id,
       s.artist_id,
       e.sessionId AS session_id,
       e.location,
       e.userAgent AS user_agent
FROM staging_events e
JOIN staging_songs s ON e.song = s.title AND e.artist = s.artist_name
WHERE e.page = 'NextSong';

Executing insert: 
INSERT INTO users (user_id, first_name, last_name, gender, level)
SELECT DISTINCT userId 